In [ ]:
import json
import os
import numpy as np
import matplotlib.pyplot as plt

from scipy.ndimage import gaussian_filter1d

from .context_manager import SpectroState
from .base_agent import BaseAgent
from .mcp_manager import MCPManager

from .utils import (
    _detect_axis_ticks, _detect_chart_border, _crop_img,
    _remap_to_cropped_canvas, _pixel_tickvalue_fitting,
    _process_and_extract_curve_points, _convert_to_spectrum,
    _find_features_multiscale, _plot_spectrum, _plot_features,
    parse_list, getenv_float, getenv_int, _load_feature_params, 
    _ROI_features_finding, merge_features, plot_merged_features, safe_to_bool, find_overlap_regions
)

In [ ]:
class SpectralRuleAnalyst(BaseAgent):
    """
    规则驱动型分析师：基于给定的物理与谱线知识进行定性分析
    """

    def __init__(self, mcp_manager: MCPManager):
        super().__init__(
            agent_name='Spectral Rule Analyst',
            mcp_manager=mcp_manager
        )

    async def describe_spectrum_picture(self, state: SpectroState):
        function_prompt = state['prompt'][f'{self.agent_name}']['describe_spectrum_picture']
        async def _filter_noise(state):
            band_name = state['band_name']
            band_wavelength = state['band_wavelength']

            if not band_name or not band_wavelength:
                return {
    "filter_noise": 'false',
    "filter_noise_wavelength": None
}
            else:
                # 找出重叠区域
                overlap_regions = find_overlap_regions(band_name, band_wavelength)
                spec = state['spectrum']
                wl = np.array(spec['new_wavelength'])
                ceiling = np.array(spec['max_unresolved_flux'])
                floor = np.array(spec['min_unresolved_flux'])
                delta = ceiling - floor

                system_prompt = function_prompt['_filter_noise']['system_prompt']
                band_name_json = json.dumps(band_name, ensure_ascii=False)
                ham = f"""
本光谱的 camera/filters 名为
{band_name_json}
下面是光谱在 camera/filters 交界区域的样本数据。
"""
                for key in overlap_regions.keys():
                    overlap = overlap_regions[key]
                    scale = overlap[1] - overlap[0]
                    scale = scale * 2
                    center = (overlap[0] + overlap[1]) / 2
                    left = center - scale / 2
                    right = center + scale / 2
                    mask = (wl >= left) & (wl <= right)
                    wl_t = wl[mask]
                    wl_t = wl_t.tolist()
                    wl_t_json = json.dumps(wl_t, ensure_ascii=False)
                    delta_t = delta[mask]
                    delta_t = delta_t.tolist()
                    delta_t_json = json.dumps(delta_t, ensure_ascii=False)

                    ham += f"""
交界区域 {key}:
波长：{wl_t_json}
Flux 误差：{delta_t_json}
"""
                user_prompt = function_prompt['_filter_noise']['user_prompt']
                user_prompt = ham + user_prompt

                response = await self.call_llm_with_context(
                    system_prompt=system_prompt,
                    user_prompt=user_prompt,
                    image_path=None,
                    parse_json=True,
                    description="Filter噪声判断"
                )
                return(response)
            
        async def _cleaning(state):
            filter_nosie = state['visual_interpretation'][0]
            if not safe_to_bool(filter_nosie.get('filter_noise', False)):
                pass
            else:
                filter_noise_wl = filter_nosie.get('filter_noise_wavelength', [])
                # filter_noise_wl_json = json.dumps(filter_noise_wl, ensure_ascii=False)
                wavelength = state['spectrum']['new_wavelength']
                flux = state['spectrum']['new_flux']
                peaks = state['merged_peaks']
                continuum = state['continuum']['flux']
                cleaned_peaks = []
                for p in peaks[:20]:
                    # index = p['rep_index']
                    wl = p['wavelength']
                    flux = p['mean_flux']
                    width_in_km_s = p['width_in_km_s']
                    width = p['width_mean']
                    wl_left = wl - width / 2
                    wl_right = wl + width / 2
                    mask = (wavelength >= wl_left) & (wavelength <= wl_right)
                    ctnm = continuum[mask]
                    ctnm = ctnm.tolist()
                    ctnm_json = json.dumps(ctnm, ensure_ascii=False)
                    flx = flux[mask]

                    distance = abs(wl - filter_noise_wl)
                    # 如果在distance中至少有一个值小于 width，则认为该峰在噪声区域内
                    if np.any(distance <= width):
                        is_artifact = True

                    system_prompt = """
你是一个天文学光谱分析专家。你的任务是识别光谱分析中可能出现的伪峰值。
使用scipy找峰算法对光谱进行识别时，可能会将两个吸收线之间的连续区域误认为峰值。
这类伪峰值通常具有以下特征：较为平坦、流量接近连续谱、峰值宽度较宽（>2000 km/s）。

请根据提供的峰值信息判断其是否为连续区域的伪峰值，并严格按以下规则输出：
- 如果是伪峰值，输出 "true"
- 如果不是伪峰值，输出 "false"
不要输出任何其他内容。
"""
                    user_prompt = f"""
请分析以下峰值信息：
- 波长：{wl} Å
- 流量：{flux}
- 宽度：{width} Å （约 {width_in_km_s} km/s）
- 峰值区域的连续谱流量列表：{ctnm_json}
- 峰值区域的流量列表：{flx}
"""
                    response = await self.call_llm_with_context(
                        system_prompt=system_prompt,
                        user_prompt=user_prompt,
                        image_path=None,
                        parse_json=True,
                        description="伪峰值判断"
                    )
                    is_continuum = safe_to_bool(response)

                    if not is_continuum and not is_artifact:
                        if p['width_in_km_s'] > 2000:
                            p['describe'] = '宽线'
                        elif p['width_in_km_s'] < 1000:
                            p['describe'] = '窄线'
                        else:
                            p['describe'] = '中等宽度'
                        cleaned_peaks.append(p)
                state['cleaned_peaks'] = cleaned_peaks
                cleaned_troughs = []
                for t in state['merged_troughs']:
                    wl = t['wavelength']
                    distance = abs(wl - filter_noise_wl)
                    if np.any(distance <= width):
                        is_artifact = True
                    if not is_artifact:
                        if t['width_in_km_s'] > 2000:
                            t['describe'] = '宽谷'
                        elif t['width_in_km_s'] < 1000:
                            t['describe'] = '窄谷'
                        else:
                            t['describe'] = '中等宽度'
                        cleaned_troughs.append(t)
                state['cleaned_troughs'] = cleaned_troughs
            return state

        async def _visual(state):
            system_prompt = function_prompt['_visual']['system_prompt']
            user_prompt_1 = function_prompt['_visual']['user_prompt_continuum']
            response_1 = await self.call_llm_with_context(
                system_prompt=system_prompt,
                user_prompt=user_prompt_1,
                image_path=state['continuum_path'],
                parse_json=True,
                description="视觉光谱定性描述——continuum"
            )

            user_prompt_2 = function_prompt['_visual']['user_prompt_lines']
            response_2 = await self.call_llm_with_context(
                system_prompt=system_prompt,
                user_prompt=user_prompt_2,
                image_path=state['spec_extract_path'],
                parse_json=True,
                description="视觉光谱定性描述"
            )

            user_prompt_3 = function_prompt['_visual']['user_prompt_quality']
            response_3 = await self.call_llm_with_context(
                system_prompt=system_prompt,
                user_prompt=user_prompt_3,
                image_path=state['spec_extract_path'],
                parse_json=True,
                description="视觉光谱定性描述"
            )
            return '\n'.join([response_1, response_2, response_3])

        # async def _get_ROI(state):
        #     _visual_json = json.dumps(state['visual_interpretation'][1], ensure_ascii=False)
        #     system_prompt = function_prompt['_get_ROI']['system_prompt']
        #     user_prompt = function_prompt['_get_ROI']['user_prompt'].format(_visual_json=_visual_json)

        #     response_2 = await self.call_llm_with_context(
        #         system_prompt=system_prompt,
        #         user_prompt=user_prompt,
        #         parse_json=True,
        #         description="视觉光谱定性描述"
        #     )
        #     return response_2

        async def _integrate(state):
            # filter_noise_json = json.dumps(state['visual_interpretation'][0], ensure_ascii=False)
            visual_json       = json.dumps(state['visual_interpretation'][1], ensure_ascii=False)
            # roi_json          = json.dumps(state['visual_interpretation'][2], ensure_ascii=False)

            system_prompt = function_prompt['_integrate']['system_prompt']
            ham = f"""
{visual_json}
"""
            user_prompt_integrate = function_prompt['_integrate']['user_prompt'] + ham
            response = await self.call_llm_with_context(
                system_prompt=system_prompt,
                user_prompt=user_prompt_integrate,
                parse_json=True,
                description="视觉光谱定性描述"
            )
            return response

        result_filter_noise = await _filter_noise(state)
        state['visual_interpretation'] = [result_filter_noise]
        await _cleaning(state)
        result_visual = await _visual(state)
        state['visual_interpretation'].append(result_visual)
        # result_ROI = await _get_ROI(state)
        # state['visual_interpretation'].append(result_ROI)
        result_integrate = await _integrate(state)
        state['visual_interpretation'] = result_integrate

        visual_interpretation_path = os.path.join(state['output_dir'], f'{state['image_name']}_visual_interpretation.txt')
        with open(visual_interpretation_path, 'w', encoding='utf-8') as f:
            json_str = json.dumps(state['visual_interpretation'], indent=2, ensure_ascii=False)
            f.write(json_str)
    
    async def preliminary_classification(self, state: SpectroState) -> str:
        """初步分类：根据光谱形态初步判断天体类型"""

        visual_interpretation_json = json.dumps(state['visual_interpretation'], ensure_ascii=False)
        sigma_list_json = json.dumps(state['sigma_list'], ensure_ascii=False)
        peaks_info = [
            {
                "wavelength": pe.get('wavelength'),
                "flux": pe.get('mean_flux'),
                "width": pe.get('width_mean'),
                "prominance": pe.get('max_prominence'),
                # "seen_in_scales_of_sigma": pe.get('seen_in_scales_of_sigma'),
            }
            for pe in state.get('merged_peaks', [])[:10]
        ]
        peak_json = json.dumps(peaks_info, ensure_ascii=False)
        trough_info = [
            {
                "wavelength": tr.get('wavelength'),
                "flux": tr.get('mean_flux'),
                "width": tr.get('width_mean'),
                # "seen_in_scales_of_sigma": tr.get('seen_in_scales_of_sigma')
            }
            for tr in state.get('merged_troughs', [])[:15]
        ]
        trough_json = json.dumps(trough_info, ensure_ascii=False)

        system_prompt = """
你是一位经验丰富的天文学光谱分析助手。

你的任务是根据光谱的定性描述和特征数据，猜测天体可能属于的类别。

可选的类别：
1. **Star**：
    - 连续谱较强，谱线通常是吸收线（如 Balmer 系列、金属线等），几乎没有明显红移。
2. **Galaxy**：
    - 有一定红移，常有发射线或吸收线。
    - 谱线通常较窄。
    - 连续谱较不明显。
    - 部分星系的连续谱呈现蓝端较低而红端显著升高的趋势。
3. **QSO**：
    - 具有**强发射线**。谱线宽度明显。
    - 连续谱覆盖可见/紫外波段。
    - 通常有明显红移。

输出要求：
- 每个猜测包含：类别、理由、置信度
- 不输出精确数值或表格
- 不尝试计算红移
- 重点在视觉与形态描述，像人类天文学家一样进行定性判断
- 不要调用工具

输出格式：
猜测 1：
- **类别**: Star / Galaxy / QSO （三选一）
- **理由**: 用简洁的语言解释分类原因（如谱线宽度、红移特征、连续谱形态）
- **置信度**: 高 / 中 / 低
猜测 2：
...
"""
        user_prompt = f"""
请根据以下光谱数据进行分析：

前一位天文学助手已经定性地描述了光谱的整体形态：
{visual_interpretation_json}
其中 filter noise 是因为在不同 filter（如 B,R,Z）重叠处出现的非物理的噪声。

在全局上，综合原曲线和 sigma={sigma_list_json} 的高斯平滑曲线，使用 scipy 函数进行了峰/谷识别。
在 ROI (region of interest) 上，综合局部的原曲线和 sigma={sigma_list_json} 的高斯平滑曲线，使用 scipy 函数进行了峰/谷识别。

关于峰/谷的讨论以以下数据为准：
- 代表性的前 10 条发射线：
{peak_json}
- 可能的吸收线：
{trough_json}

请根据这些描述和数据，猜测该光谱可能属于哪一类或几类天体。
"""
        response = await self.call_llm_with_context(
            system_prompt = system_prompt,
            user_prompt = user_prompt,
            image_path=state['image_path'],
            parse_json=True,
            description="初步分类"
        )
        state['preliminary_classification'] = response

    async def preliminary_classification_monkey(self, state):
        """ My dear monkey friend and its typewriter """
        preliminary_classification_json = json.dumps(state['preliminary_classification'], ensure_ascii=False)
        prompt = f"""
你是一个天文学光谱分析助手。
你接收到的是其他助手对一张光谱的光源类别的初步猜测：
{preliminary_classification_json}

请输出这份猜测里给出的光源类别。

输出格式为数组 List[str]，数组的元素必须在 "Star", "Galaxy" 和 "QSO" 中选择。

- 注意：即使只有一个满足条件的光源类别，也要以 List[str] 的格式输出。
"""
        response = await self.call_llm_with_context(
            system_prompt = '',
            user_prompt = prompt,
            parse_json=True,
            description="初步分类猴子"
        )
        return response
    ###################################
    # QSO part
    ###################################
    async def _QSO(self, state):
        """QSO"""
        def _common_prompt_header_QSO(state, include_rule_analysis=True, include_step_1_only=False):
            """构造每个 step 公共的 prompt 前段"""
            visual_json = json.dumps(state['visual_interpretation'], ensure_ascii=False)
            # peak_json = json.dumps(state['peaks'][:10], ensure_ascii=False)
            # trough_json = json.dumps(state['troughs'], ensure_ascii=False)
            peaks_info = [
                {
                    "wavelength": pe.get('wavelength'),
                    "flux": pe.get('mean_flux'),
                    "width": pe.get('width_mean'),
                    "prominance": pe.get('max_prominence'),
                    "seen_in_scales_of_sigma": pe.get('seen_in_scales_of_sigma'),
                }
                for pe in state.get('merged_peaks', [])[:10]
            ]
            peak_json = json.dumps(peaks_info, ensure_ascii=False)
            trough_info = [
                {
                    "wavelength": tr.get('wavelength'),
                    "flux": tr.get('mean_flux'),
                    "width": tr.get('width_mean'),
                    "seen_in_scales_of_sigma": tr.get('seen_in_scales_of_sigma')
                }
                for tr in state.get('merged_troughs', [])[:15]
            ]
            trough_json = json.dumps(trough_info, ensure_ascii=False)

            header = f"""
    你是一位天文学光谱分析助手。

    以下信息可能来自于一个未知红移的 QSO 光谱。

    之前的助手已经对这个光谱进行了初步描述：
    {visual_json}

    该光谱的波长范围是{state['spectrum']['new_wavelength'][0]} Å 到 {state['spectrum']['new_wavelength'][-1]} Å。
    """

            if include_rule_analysis and state['rule_analysis_QSO']:
                if include_step_1_only==True:
                    rule_json = json.dumps(state['rule_analysis_QSO'][0], ensure_ascii=False)
                else:
                    rule_json = json.dumps("\n".join(str(item) for item in state['rule_analysis_QSO']), ensure_ascii=False)
                header += f"\n之前的助手已经进行了一些分析:\n{rule_json}\n"

            tol_pixels = getenv_int("TOL_PIXELS", 10)
            a_x = state['pixel_to_value']['x']['a']
            tol_wavelength = a_x * tol_pixels
            header += f"""
    综合原曲线和 sigma={state['sigma_list']} 的高斯平滑曲线，使用 scipy 函数进行了峰/谷识别。
    关于峰/谷的讨论以以下数据为准：
    - 代表性的前 10 条发射线：
    {peak_json}
    - 可能的吸收线：
    {trough_json}
    - 波长误差在 ~ ±{tol_wavelength/2} Å 的量级或更大
    """
            return header

        def _common_prompt_tail(step_title, extra_notes=""):
            """构造每个 step 公共尾部，保留 step 特有输出/分析指示"""
            tail = f"""
    ---

    输出格式为：
    {step_title}
    ...

    ---

    🧭 注意：
    - 计算得来的非原始数据，输出时保留 3 位小数。
    - 不需要进行重复总结。
    - 不需要逐行地重复输入数据；
    - 重点在物理推理与合理解释；
    - 请保证最终输出完整，不要中途截断。
    """
            if extra_notes:
                tail = extra_notes + "\n" + tail
            return tail
    
        async def step_1_QSO(state):
            header = _common_prompt_header_QSO(state, include_rule_analysis=False)
            tail = _common_prompt_tail("Step 1: Lyα 谱线检测")

            prompt = header + """
请按以下步骤分析:

Step 1: Lyα 谱线检测
假设该光谱中存在 Lyα 发射线（λ_rest = 1216 Å）：
1. 在光谱蓝端，流量较大，且有一定宽度的峰中，推测哪条最可能为 Lyα 线（从提供的峰列表中选择）。
2. 输出：
- 观测波长 λ_obs
- 流量 Flux
- 谱线宽度
3. 使用工具 calculate_redshift 计算该峰为 Lyα 发射线时的红移 z。
4. 检查蓝端（短波长方向）是否存在 Lyα forest 特征：吸收线相对更密集、较窄且分布在 Lyα 蓝端附近。请指出并进行简短说明。
""" + tail
            
            response = await self.call_llm_with_context(
                system_prompt='', 
                user_prompt=prompt, 
                parse_json=True, 
                description="Step 1 Lyα 分析"
            )
            state['rule_analysis_QSO'].append(response)

        async def step_2_QSO(state):
            header = _common_prompt_header_QSO(state)
            tail = _common_prompt_tail("Step 2: 其他显著发射线分析")

            prompt = header + """
请继续分析:

Step 2: 其他显著发射线分析
1. 在 Step 1 得到的红移下，使用工具 predict_obs_wavelength 计算以下三条主要发射线：C IV 1549, C III] 1909, Mg II 2799 在光谱中的理论位置。
2. 光谱中是否有与三者相匹配的峰？
3. 如果存在发射线与观测峰值的匹配，根据匹配结果，分别使用工具 calculate_redshift 计算红移。按“发射线名--静止系波长--观测波长--红移”的格式输出。
""" + tail

            response = await self.call_llm_with_context('', prompt, parse_json=False, description="Step 2 发射线分析")
            state['rule_analysis_QSO'].append(response)

        async def step_3_QSO(state):
            header = _common_prompt_header_QSO(state)
            tail = _common_prompt_tail("Step 3: 综合判断")

            prompt = header + """
请继续分析:

Step 3: 综合判断
1. 在 Step 1 到 Step 2 中，如果：
    - C IV 和 C III] 两条主要谱线存在缺失或大幅偏移
    - 使用 lyα 谱线计算的红移与其他谱线的计算结果不一致，
此时请输出“应优先假设 Lyα 谱线未被找峰程序捕获”，并结束 Step 3 的分析。不要输出其他信息。
2.仅在有显著的 Lyα 峰值，且红移计算结果与其他谱线基本一致时，进行以下操作：
    - 因为天文学中存在外流等现象，请将当前所有匹配中**最低电离态谱线的红移**作为光谱的红移。输出红移结果。（因为存在不对称和展宽，Lyα的置信度是较低的）
""" + tail

            response = await self.call_llm_with_context('', prompt, parse_json=False, description="Step 3 综合判断")
            state['rule_analysis_QSO'].append(response)
            
        async def step_4_QSO(state):
            header = _common_prompt_header_QSO(state, include_step_1_only=True)
            tail = _common_prompt_tail("Step 4: 补充步骤（假设 Step 1 所选择的谱线并非 Lyα）")

            prompt = header + """
请继续分析:

Step 4: 补充步骤（假设 Step 1 所选择的谱线并非 Lyα）
- 请抛开前述步骤的分析内容。考虑 Step 1 所选择的谱线实际上是除 Lyα 外的其他主要发射线。
    - 假设该峰值可能对应的谱线为 C IV：
        - 输出该峰对应谱线的信息：
            - 观测波长 λ_obs
            - 流量 Flux
            - 谱线宽度
            - 根据 λ_rest，使用工具 calculate_redshift 初步计算红移 z
        - 使用工具 predict_obs_wavelength 计算在此红移下的其他主要发射线（如 C III] 和 Mg II）的理论位置。光谱中是否有与它们匹配的发射线？
        - 如果存在可能的发射线-观测波长匹配结果，使用工具 calculate_redshift 计算它们的红移。按照“发射线名--静止系波长--观测波长--红移”的格式进行输出
    
    - 若以上假设不合理，则假设该峰值可能对应 C III] 等其他主要谱线，重复推断。

    - 综合 Step 4 的所有分析，给出：
        - **最低电离态谱线的红移** 作为光谱红移
        - 输出 “发射线名--静止系波长--观测波长--红移” 匹配

- 注意：允许在由于光谱边缘的信号残缺或信噪比不佳导致部分发射线不可见。   
- 抛开其他步骤的分析内容，本节的判断是否支持 Lyα 谱线未被找峰程序捕获的假设？
""" + tail

            response = await self.call_llm_with_context('', prompt, parse_json=False, description="Step 4 补充分析")
            state['rule_analysis_QSO'].append(response)
        
        await step_1_QSO(state)
        await step_2_QSO(state)
        await step_3_QSO(state)
        await step_4_QSO(state)

#     # --------------------------
#     # Run 全流程
#     # --------------------------
    async def run(self, state: SpectroState):
        """执行规则分析完整流程"""
        try:
            await self.describe_spectrum_picture(state)
            ROI_peaks, ROI_troughs = _ROI_features_finding(state)
            # print(f"ROI_peaks:\n{ROI_peaks}")
            # print(f"ROI_troughs:\n{ROI_troughs}")
            state['merged_peaks'], state['merged_troughs'] = merge_features(
                global_peaks=state['peaks'],
                global_troughs=state['troughs'],
                ROI_peaks=ROI_peaks,
                ROI_troughs=ROI_troughs, 
                tol_pixels=10,
            )
            
            plot_merged_features(state)
            
            await self.preliminary_classification(state)
            # print(state['preliminary_classification'])

            _shakespear = await self.preliminary_classification_monkey(state)
            state['possible_object'] = _shakespear
            # print(f"Monkeys types: {_shakespear}")

            if "QSO" in _shakespear:
                await self._QSO(state)
                # await self.step_1_QSO(state)
                # await self.step_2_QSO(state)
                # await self.step_3_QSO(state)
                # await self.step_4_QSO(state)
            # if "Galaxy" in _shakespear:
            #     await self.step_1_galaxy(state)
            #     await self.step_2_galaxy(state)
            #     await self.step_3_galaxy(state)
            #     await self.step_4_galaxy(state)
            #     await self.step_5_galaxy(state)
            return state
        except Exception as e:
            import traceback
            print("❌ An error occurred during spectral analysis:")
            print(f"Error type: {type(e).__name__}")
            print(f"Error message: {str(e)}")
            print("Full traceback:")
            traceback.print_exc()
            # 可选：返回当前状态或抛出异常
            raise  # 如果你希望调用者也能捕获该异常
   